## Overview

For my final Codecademy data analysis project, I wanted to work with data about dinosaurs. In this project, I used data from the 2020 publication *Ten more years of discovery: revisiting the quality of the sauropodomorph dinosaur fossil record* available on [Dryad](https://datadryad.org/stash/dataset/doi:10.5061%2Fdryad.6hdr7sqxb).

This dataset collects information regarding 305 individual species of sauropodomorphs (long-necked dinosaurs and their relatives), including geographic location, size, and age of fossil. The information comes from 1144 individual specimens, and the paper is mainly looking at how complete each fossil is. 

For my analysis, I'll mostly be looking at the summary information as well as the information on each individual fossil. The initial questions I'll try to answer are:

- How did size and diversity of sauropodomorphs change throughout the Mezozoic Era?
- Which clades of sauropodomorphs do we have the most fossil specimens of?
- Has the rate of discovery of species increased or decreased since we identified the first sauropodomorph?
- Where did the oldest sauropodomorphs live, and how did they spread across the globe?
- What bones are the most likely to fossilize, based on the fossil specimens we have found so far?

## Importing the Data

There are multiple csv files included in the dataset. I will be using two: the summary data and the SCM2 data (which measures specimen completeness).

I'll start by importing some common libraries which will be useful for analyzing and visualizing the data.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
summary_df = pd.read_csv('Sauropod_Summary.csv')
completeness_df = pd.read_csv('Sauropod_SCM2.csv')

I had to do some initial cleanup of the data in Excel before importing - I removed the top summary line, which divided the data into sections, and changed the encoding to UTF-8. Now I'll take a look at the data.

In [3]:
summary_df.head()

,Clade 1,Clade 2,Clade 3,Clade 4,Clade 5,Clade 6,Clade 7,Clade 8,Clade 9,Clade 10,...,Classification,Primary lithology,Setting 1,Setting 2,Setting 3,Setting 4,Coarse category,Eliason et al. (2017) category,M&U category,Unnamed: 48
0,Sauropodomorpha,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Sauropodomorpha,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Siliclastic,Mudstone,Fluvial channel,NaN,NaN,NaN,Fluvio-lacustrine,Fluvial,Inland,-
2,Sauropodomorpha,Massopoda,Massospondylidae,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Siliclastic,Sandstone,Lacustrine,NaN,NaN,NaN,Fluvio-lacustrine,Lacustrine,Inland,-
3,Sauropodomorpha,Massopoda,Sauropodiformes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Siliclastic,Sandstone,Fluvial channel,Lacustrine,NaN,NaN,Fluvio-lacustrine,Fluvial,Inland,-
4,Sauropodomorpha,Massopoda,Sauropodiformes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Siliclastic,Sandstone,Fluvial channel,Lacustrine,NaN,NaN,Fluvio-lacustrine,Fluvial,Inland,-


In [4]:
completeness_df.head()

,Partitioning data,Previous Mannion completeness score,"Inclusion reasons: New [N]; New sauropodomorph, old specimen [Nos]; New specimens [ns]; Species-level rescore [S]; Reinput [R]",Clade 1,Clade 2,Clade 3,Clade 4,Clade 5,Clade 6,Clade 7,...,Ilia,Pubes,Ischia,Femora,Tibiae,Fibulae,Tarsals,Metatarsals,Pes.digits,TOTAL
0,-,NaN,NaN,Sauropodomorpha,NaN,NaN,NaN,NaN,NaN,NaN,...,2,2,2,4,2,2,1,1.5,1.5,100
1,NaN,NaN,N,Sauropodomorpha,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0.1
2,NaN,NaN,N,Sauropodomorpha,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0.25,0,0,0,0,3.583333333
3,NaN,NaN,N,Sauropodomorpha,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,10.1875
4,NaN,NaN,N,Sauropodomorpha,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0.1875


As expected from my initial overview of the data, this is quite messy. There are also a lot of columns that I won't need for my analysis. The next step is to review the data and identify any duplicates or null values before I clean it up.

In [5]:
summary_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 312 entries, 0 to 311
Data columns (total 49 columns):
 #   Column                                                                                                                                                                                                                                                                                                                                                                 Non-Null Count  Dtype  
---  ------                                                                                                                                                                                                                                                                                                                                                                 --------------  -----  
 0   Clade 1                                                                                                           

In [6]:
completeness_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1144 entries, 0 to 1143
Data columns (total 54 columns):
 #   Column                                                                                                                          Non-Null Count  Dtype 
---  ------                                                                                                                          --------------  ----- 
 0   Partitioning data                                                                                                               356 non-null    object
 1   Previous Mannion completeness score                                                                                             220 non-null    object
 2   Inclusion reasons: New [N]; New sauropodomorph, old specimen [Nos]; New specimens [ns]; Species-level rescore [S]; Reinput [R]  1139 non-null   object
 3   Clade 1                                                                                                     

Both dataframes have some null values and missing data. I'm not going to do anything about this right now, but I will need to keep it in mind when cleaning the data.

Now I will check for duplicate rows.

In [7]:
summary_duplicates = summary_df[summary_df.duplicated()]
summary_duplicates.head()

,Clade 1,Clade 2,Clade 3,Clade 4,Clade 5,Clade 6,Clade 7,Clade 8,Clade 9,Clade 10,...,Classification,Primary lithology,Setting 1,Setting 2,Setting 3,Setting 4,Coarse category,Eliason et al. (2017) category,M&U category,Unnamed: 48


In [8]:
complete_duplicates = completeness_df[completeness_df.duplicated()]
complete_duplicates.head()

,Partitioning data,Previous Mannion completeness score,"Inclusion reasons: New [N]; New sauropodomorph, old specimen [Nos]; New specimens [ns]; Species-level rescore [S]; Reinput [R]",Clade 1,Clade 2,Clade 3,Clade 4,Clade 5,Clade 6,Clade 7,...,Ilia,Pubes,Ischia,Femora,Tibiae,Fibulae,Tarsals,Metatarsals,Pes.digits,TOTAL


Okay, there are no duplicate rows in either dataframe. Next I want to clean up the data to remove irrelevant information and make the data easier to analyze.

## Cleaning the Summary Data

I'll get started by going through all the columns in each dataset and removing the ones I won't need for analysis. This will make the data easier to work with. First, I need a list of all the columns in the summary dataset. Before I clean up the data, I'm going to make a reference copy that I can go back to if I accidentally delete anything important.

In [9]:
backup_summary_df = summary_df

In [10]:
#Get a list of all the column names
list(summary_df)

['Clade 1',
 'Clade 2',
 'Clade 3',
 'Clade 4',
 'Clade 5',
 'Clade 6',
 'Clade 7',
 'Clade 8',
 'Clade 9',
 'Clade 10',
 'Clade 11',
 'Clade 12',
 'Clade 13',
 'Clade 14',
 'Clade 15',
 'Taxa',
 'valid species (v); valid genera (vg); indeterminate (i)',
 "Data derived: M&U [Mannion and Upchurch (2010) dataset]; R [Revised specimen ident. since M&U or species excluded in M&U, but no completeness calculation needed as specimen scores provided in Mannion's database]; N/R [New taxa, reinput taxa, additional specimens for taxa, revised specimen ident. etc since M&U, with completeness calculated in this dataset]",
 'Year',
 'SCM2',
 'Hemisphere',
 'Continent',
 'Country',
 'Modern latitude',
 'Palaeolatitude',
 'Group',
 'Formation',
 'Member',
 'Period',
 'Epoch',
 'Stage 1',
 'Stage 2',
 'Stage no',
 'Max_ma',
 'Min_ma',
 'Conservation',
 'Concentration',
 'Body Mass (Benson et al. 2018)',
 'Facies',
 'Classification',
 'Primary lithology',
 'Setting 1',
 'Setting 2',
 'Setting 3',
 'Sett

I've identified the following columns I can get rid of by also reviewing the data in Excel:

- valid species (I only want to work with valid species, so I'll remove any rows where the species isn't valid and then remove the column
- Data derived
- Group
- Formation
- Member
- Stage 1
- Stage 2
- Stage no
- Conservation
- Concentration
- All columns after Body Mass

The clade and taxa data I need to keep, but perhaps I can combine some of the columns later. First, I'll remove these unneccessary columns.

In [11]:
#Remove non-valid species rows from the dataframe
summary_df = summary_df[summary_df['valid species (v); valid genera (vg); indeterminate (i)'] == 'v']

In [12]:
#Remove unnecessary columns
summary_df = summary_df.loc[:, ~summary_df.columns.str.contains('^Unnamed')]
summary_df = summary_df.drop(summary_df.columns[[16, 17, 25, 26, 27, 30, 31, 32, 35, 36, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47]], axis = 1)

In [13]:
#Get list of remaining columns
list(summary_df)

['Clade 1',
 'Clade 2',
 'Clade 3',
 'Clade 4',
 'Clade 5',
 'Clade 6',
 'Clade 7',
 'Clade 8',
 'Clade 9',
 'Clade 10',
 'Clade 11',
 'Clade 12',
 'Clade 13',
 'Clade 14',
 'Clade 15',
 'Taxa',
 'Year',
 'SCM2',
 'Hemisphere',
 'Continent',
 'Country',
 'Modern latitude',
 'Palaeolatitude',
 'Period',
 'Epoch',
 'Max_ma',
 'Min_ma',
 'Body Mass (Benson et al. 2018)']

That's better, but the list of clades is still very long and messy. The Taxa column contains genus and species, but I will want some higher-level classifications for my analysis. 

Let's go down the clades by column and see what information they have.

In [15]:
#Get list of total species
print(len(summary_df))

305


In [16]:
#Get the number of rows with values for Clade 1
print(summary_df['Clade 1'].value_counts())

Sauropodomorpha    305
Name: Clade 1, dtype: int64


Since all the rows in the Clade 1 column have the same value and there are no null values, we can remove it.

In [17]:
#Get the number of rows with values for Clade 2
print(summary_df['Clade 2'].value_counts())
print(summary_df['Clade 2'].isna().sum())

Massopoda         277
Plateosauridae      6
Guaibasauridae      2
Unaysauridae        2
Name: Clade 2, dtype: int64
18


There are 4 values in Clade 2. Most rows have the value Massopoda. According to [DinoChecker.com](http://www.dinochecker.com/dinosaurfaqs/what-is-massopoda), Massopoda and the other values in this column are all branches of the sauropodomorph family tree. According to the tree diagram on the website, dinosaurs in the Massopoda category eventually gave rise to true sauropods, but we're not there yet. 

There are also some null values in Clade 2. These are likely fossils that don't fit into a more specific clade. When categorizing them, I'll want to keep them as Sauropodomorpha.

In [18]:
#Get the number of rows with values for Clade 3
print(summary_df['Clade 3'].value_counts())
print(summary_df['Clade 3'].isna().sum())

Sauropodiformes     265
Massospondylidae      9
Riojasauridae         3
Name: Clade 3, dtype: int64
28


Clade 3 has more null values than Clade 2, and three distinct non-null values. These categories all appear to be branches leading to true sauropods. I've decided I want my first category distinction to be between true sauropods and sauropodomorphs, so I'm looking for a clade where "Sauropod" first appears.

In [19]:
#Get the number of rows with values for Clade 4
print(summary_df['Clade 4'].value_counts())
print(summary_df['Clade 4'].isna().sum())

Sauropoda    251
Name: Clade 4, dtype: int64
54


Okay, here is the Sauropod clade I was looking for. I'm going to create a new column that notes whether a species is a sauropod or a sauropodomorph, and then remove the columns for clades 1-4. 

In [20]:
#Create a new column to distinguish between sauropodomorphs and sauropods
summary_df['Sauropod'] = summary_df['Clade 4'].notnull()
#Remove columns for Clades 1-4
summary_df = summary_df.drop(summary_df.columns[[0,1,2,3]], axis = 1)
summary_df.tail()

,Clade 5,Clade 6,Clade 7,Clade 8,Clade 9,Clade 10,Clade 11,Clade 12,Clade 13,Clade 14,...,Continent,Country,Modern latitude,Palaeolatitude,Period,Epoch,Max_ma,Min_ma,Body Mass (Benson et al. 2018),Sauropod
307,Eusauropoda,Neosauropoda,Macronaria,Camarasauromorpha,Titanosauriformes,Somphospondyli,Titanosauria,NaN,NaN,NaN,...,Africa,Madagascar,-15.907778,-36.57,Cretaceous,Late,72.1,66.0,NaN,True
308,Eusauropoda,Neosauropoda,Macronaria,Camarasauromorpha,Titanosauriformes,Somphospondyli,Titanosauria,Eutitanosauria,Lithostrotia,NaN,...,Europe,Russia,54.433331,48.10,Cretaceous,Early,132.9,130.0,NaN,True
309,Eusauropoda,Neosauropoda,Macronaria,Camarasauromorpha,Titanosauriformes,Somphospondyli,Titanosauria,NaN,NaN,NaN,...,Asia,China,34.100000,37.40,Cretaceous,Late,125.0,100.5,16164.17118,True
310,Eusauropoda,Neosauropoda,Macronaria,Camarasauromorpha,Titanosauriformes,Somphospondyli,Titanosauria,NaN,NaN,NaN,...,Asia,China,35.940830,32.11,Cretaceous,Early,129.4,113.0,NaN,True
311,Eusauropoda,Neosauropoda,Macronaria,Camarasauromorpha,Titanosauriformes,Somphospondyli,Titanosauria,Eutitanosauria,Lithostrotia,NaN,...,Asia,China,37.500000,37.90,Cretaceous,Late,83.6,72.1,NaN,True


Okay, let's continue with Clade 5.

In [21]:
#Get the number of rows with values for Clade 5
print(summary_df['Clade 5'].value_counts())
print(summary_df['Clade 5'].isna().sum())

Eusauropoda        231
Lessemsauridae       3
Vulcanodontidae      2
Name: Clade 5, dtype: int64
69


The Eusauropoda appear to be the group that give rise to the most well known sauropods like Diplodocus and Titanosaurus. According to [this chart](http://www.dinochecker.com/dinosaurfaqs/what-is-eusauropoda) the next big split is between Diplodoid sauropods and Macronaria dinosaurs. I'll split the data there.

In [22]:
#Get the number of rows with values for Clade 6
print(summary_df['Clade 6'].value_counts())
print(summary_df['Clade 6'].isna().sum())

Neosauropoda        185
Mamenchisauridae     20
Turiasauria           4
Cetiosauridae         3
Name: Clade 6, dtype: int64
93


In [23]:
#Get the number of rows with values for Clade 7
print(summary_df['Clade 7'].value_counts())
print(summary_df['Clade 7'].isna().sum())

Macronaria       138
Diplodocoidea     47
Name: Clade 7, dtype: int64
120


Okay, the split happens at Clade 7. I'll create a new column that differentiates whether the species is Macronaria, Diplodocoidea, or a more basal sauropod.

In [24]:
#Create a new column for Main Clade
summary_df['Main Clade'] = \
summary_df.apply(lambda row: 'Sauropodomorph' if row['Sauropod'] == False \
                                            else('Sauropod' if pd.isnull(row['Clade 7']) else row['Clade 7']), axis = 1)

In [25]:
summary_df.head()

,Clade 5,Clade 6,Clade 7,Clade 8,Clade 9,Clade 10,Clade 11,Clade 12,Clade 13,Clade 14,...,Country,Modern latitude,Palaeolatitude,Period,Epoch,Max_ma,Min_ma,Body Mass (Benson et al. 2018),Sauropod,Main Clade
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,South Africa,-28.466389,-42.64,Jurassic,Early,201.3,190.8,NaN,False,Sauropodomorph
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Argentina,-30.500000,-33.25,Jurassic,Early,201.3,174.1,39.600177,False,Sauropodomorph
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,USA,41.809765,23.50,Jurassic,Early,201.3,190.8,235.601348,False,Sauropodomorph
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,USA,42.100000,23.80,Jurassic,Early,201.3,190.8,NaN,False,Sauropodomorph
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,South Africa,-28.466667,-42.64,Jurassic,Early,201.3,190.8,NaN,False,Sauropodomorph


In [26]:
#Drop columns for Clades 5-7
summary_df = summary_df.drop(summary_df.columns[[0,1,2]], axis = 1)

Now that I've dropped up to Clade 8, I'll continue with my analysis.

Since I've kept one of the divisions, I'm going to compare several clades together to get a better picture of how things branch from here.

In [27]:
#Get the number of rows with values for Clades 8-10
print(summary_df[['Main Clade', 'Clade 8', 'Clade 9', 'Clade 10']].value_counts())

Main Clade     Clade 8            Clade 9            Clade 10       
Macronaria     Camarasauromorpha  Titanosauriformes  Somphospondyli     112
                                                     Brachiosauridae     10
Diplodocoidea  Flagellicaudata    Diplodocidae       Diplodocinae         7
               Rebbachisauridae   Khebbashia         Limaysaurinae        5
               Flagellicaudata    Diplodocidae       Apatosaurinae        5
               Rebbachisauridae   Khebbashia         Nigersaurinae        4
dtype: int64


Now I'm starting to see some familiar terms. The Diplodocoidea seem to be pretty well categorized, so I'll keep the categories for Clade 10 for them. For the Macronaria, there's still a pretty big group with Somphospondyli that needs to be broken down. 

In [28]:
#Create column for Subclade
summary_df['Subclade'] = summary_df.apply(
    lambda row: row['Main Clade'] if pd.isnull(row['Clade 10']) else row['Clade 10'], axis = 1)

In [29]:
#Drop columns for Clades 8-10
summary_df = summary_df.drop(summary_df.columns[[0,1,2]], axis = 1)

In [30]:
summary_df.head()

,Clade 11,Clade 12,Clade 13,Clade 14,Clade 15,Taxa,Year,SCM2,Hemisphere,Continent,...,Modern latitude,Palaeolatitude,Period,Epoch,Max_ma,Min_ma,Body Mass (Benson et al. 2018),Sauropod,Main Clade,Subclade
1,NaN,NaN,NaN,NaN,NaN,Aardonyx celestae,2010.0,32.673313,S,Africa,...,-28.466389,-42.64,Jurassic,Early,201.3,190.8,NaN,False,Sauropodomorph,Sauropodomorph
2,NaN,NaN,NaN,NaN,NaN,Adeopapposaurus mognai,2009.0,91.525000,S,South America,...,-30.500000,-33.25,Jurassic,Early,201.3,174.1,39.600177,False,Sauropodomorph,Sauropodomorph
3,NaN,NaN,NaN,NaN,NaN,Ammosaurus major,1891.0,43.220083,N,North America,...,41.809765,23.50,Jurassic,Early,201.3,190.8,235.601348,False,Sauropodomorph,Sauropodomorph
4,NaN,NaN,NaN,NaN,NaN,Anchisaurus polyzelus,1885.0,44.055667,N,North America,...,42.100000,23.80,Jurassic,Early,201.3,190.8,NaN,False,Sauropodomorph,Sauropodomorph
5,NaN,NaN,NaN,NaN,NaN,Arcusaurus pereirabdalorum,2011.0,3.387333,S,Africa,...,-28.466667,-42.64,Jurassic,Early,201.3,190.8,NaN,False,Sauropodomorph,Sauropodomorph


Almost there - now I'm going to review clades 11-15 the same way.

In [31]:
#Get the number of rows with values for Clades 11 and 12
print(summary_df[['Subclade', 'Clade 11', 'Clade 12']].value_counts())

Subclade        Clade 11      Clade 12      
Somphospondyli  Titanosauria  Eutitanosauria    61
dtype: int64


In [32]:
#Get the number of rows with values for Clades 12 and 13
print(summary_df[['Clade 12', 'Clade 13']].value_counts())

Clade 12        Clade 13    
Eutitanosauria  Lithostrotia    40
                Lognkosauria     9
                Aeolosaurini     8
                Rinconsauria     2
dtype: int64


It looks like the Somphospondyli group can largely be split into titanousaurs and non-titanosaurs. These are pretty big groups, but I think going further than that will make things too granular. I'm going to revise the Subclade column to break out Somphospondyli into Titanosauria and Somphospondyli and then remove the rest of the clade columns.

In [33]:
#Add new values to Subclade column
summary_df['Subclade'] = summary_df.apply(
    lambda row: row['Subclade'] if pd.isnull(row['Clade 11']) else row['Clade 11'], axis = 1)

In [34]:
#Double-check values for Subclade
print(summary_df['Subclade'].value_counts())

Titanosauria       82
Sauropod           66
Sauropodomorph     54
Diplodocoidea      26
Somphospondyli     24
Macronaria         15
Brachiosauridae     9
Euhelopodidae       7
Diplodocinae        7
Apatosaurinae       5
Limaysaurinae       5
Nigersaurinae       4
?                   1
Name: Subclade, dtype: int64


Not happy about the question mark, so let's investigate that.

In [35]:
#Print the row with the question mark
print(summary_df[summary_df.Subclade == '?'])

    Clade 11 Clade 12 Clade 13 Clade 14 Clade 15                  Taxa  \
191        ?      NaN      NaN      NaN      NaN  Europasaurus holgeri   

       Year  SCM2 Hemisphere Continent  ... Modern latitude  Palaeolatitude  \
191  2006.0  73.0          N    Europe  ...       51.900002           35.24   

       Period Epoch Max_ma  Min_ma  Body Mass (Benson et al. 2018)  Sauropod  \
191  Jurassic  Late  157.3   152.1                     1045.226583      True   

     Main Clade Subclade  
191  Macronaria        ?  

[1 rows x 21 columns]


I'd like to go back to the earlier clades to try and classify this species, so I'll refer to my backup summary dataframe.

In [37]:
#Print row with question mark from backup data
backup_summary_df[backup_summary_df['Clade 11'] == '?']

,Clade 1,Clade 2,Clade 3,Clade 4,Clade 5,Clade 6,Clade 7,Clade 8,Clade 9,Clade 10,...,Classification,Primary lithology,Setting 1,Setting 2,Setting 3,Setting 4,Coarse category,Eliason et al. (2017) category,M&U category,Unnamed: 48
191,Sauropodomorpha,Massopoda,Sauropodiformes,Sauropoda,Eusauropoda,Neosauropoda,Macronaria,Camarasauromorpha,Titanosauriformes,Brachiosauridae,...,Siliclastic,Mudstone,Coastal,NaN,NaN,NaN,Coastal,Estuarine / near shore,Coastal,-


In [38]:
#Replace question mark with 'Brachiosauridae', the value in Clade 10
summary_df.loc[summary_df['Subclade'] == '?', 'Subclade'] = 'Brachiosauridae'

In [39]:
#Check values again
print(summary_df['Subclade'].value_counts())

Titanosauria       82
Sauropod           66
Sauropodomorph     54
Diplodocoidea      26
Somphospondyli     24
Macronaria         15
Brachiosauridae    10
Euhelopodidae       7
Diplodocinae        7
Apatosaurinae       5
Limaysaurinae       5
Nigersaurinae       4
Name: Subclade, dtype: int64


In [40]:
#Drop remaining Clade columns
summary_df = summary_df.drop(summary_df.columns[[0,1,2,3,4]], axis = 1)

In [41]:
#Get dataframe info
summary_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 305 entries, 1 to 311
Data columns (total 16 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   Taxa                            305 non-null    object 
 1   Year                            305 non-null    float64
 2   SCM2                            305 non-null    float64
 3   Hemisphere                      305 non-null    object 
 4   Continent                       305 non-null    object 
 5   Country                         305 non-null    object 
 6   Modern latitude                 305 non-null    float64
 7   Palaeolatitude                  305 non-null    float64
 8   Period                          305 non-null    object 
 9   Epoch                           304 non-null    object 
 10  Max_ma                          305 non-null    float64
 11  Min_ma                          305 non-null    float64
 12  Body Mass (Benson et al. 2018)  156 

That's a lot better, only 15 columns. I can also drop the Sauropod column now that I have the Main Clade and Subclade columns. The only remaining non-null data is the estimated body mass, which is missing for some of the species. That's fine with me. 

Before I move onto the completeness dataset, I want to reorder and rename the columns to make them easier to work with.

In [42]:
#Get list of column names
columns = list(summary_df)
columns

['Taxa',
 'Year',
 'SCM2',
 'Hemisphere',
 'Continent',
 'Country',
 'Modern latitude',
 'Palaeolatitude',
 'Period',
 'Epoch',
 'Max_ma',
 'Min_ma',
 'Body Mass (Benson et al. 2018)',
 'Sauropod',
 'Main Clade',
 'Subclade']

In [47]:
#Reorder column names
new_columns = [ 'Sauropod',
 'Main Clade',
 'Subclade',
 'Taxa',
 'Year',
 'SCM2',
 'Hemisphere',
 'Continent',
 'Country',
 'Modern latitude',
 'Palaeolatitude',
 'Period',
 'Epoch',
 'Max_ma',
 'Min_ma',
 'Body Mass (Benson et al. 2018)',
]

In [48]:
#Reorder columns in dataframe
summary_df = summary_df[new_columns]

KeyError: "['Sauropod'] not in index"

In [45]:
#Drop "Sauropod" column
summary_df = summary_df.drop(summary_df.columns[0], axis = 1)

In [49]:
#Rename columns
renamed_columns = [ 'main_clade',
 'sub_clade',
 'taxa',
 'year_found',
 'completeness',
 'hemisphere',
 'continent',
 'country',
 'mod_lat',
 'paleo_lat',
 'period',
 'epoch',
 'max_ma',
 'min_ma',
 'body_mass_est',
]

In [50]:
summary_df.columns = renamed_columns
summary_df.head()

,main_clade,sub_clade,taxa,year_found,completeness,hemisphere,continent,country,mod_lat,paleo_lat,period,epoch,max_ma,min_ma,body_mass_est
1,Sauropodomorph,Sauropodomorph,Aardonyx celestae,2010.0,32.673313,S,Africa,South Africa,-28.466389,-42.64,Jurassic,Early,201.3,190.8,NaN
2,Sauropodomorph,Sauropodomorph,Adeopapposaurus mognai,2009.0,91.525000,S,South America,Argentina,-30.500000,-33.25,Jurassic,Early,201.3,174.1,39.600177
3,Sauropodomorph,Sauropodomorph,Ammosaurus major,1891.0,43.220083,N,North America,USA,41.809765,23.50,Jurassic,Early,201.3,190.8,235.601348
4,Sauropodomorph,Sauropodomorph,Anchisaurus polyzelus,1885.0,44.055667,N,North America,USA,42.100000,23.80,Jurassic,Early,201.3,190.8,NaN
5,Sauropodomorph,Sauropodomorph,Arcusaurus pereirabdalorum,2011.0,3.387333,S,Africa,South Africa,-28.466667,-42.64,Jurassic,Early,201.3,190.8,NaN


## Cleaning the Completeness Data

Like the summary data, I'll get started by making a copy of the completeness dataframe and then examining the columns to see which I can remove.

In [51]:
#Create a backup dataframe
backup_completeness_df = completeness_df

In [52]:
#List column names
list(completeness_df)

['Partitioning data',
 'Previous Mannion completeness score',
 'Inclusion reasons: New [N]; New sauropodomorph, old specimen [Nos]; New specimens [ns]; Species-level rescore [S]; Reinput [R]',
 'Clade 1',
 'Clade 2',
 'Clade 3',
 'Clade 4',
 'Clade 5',
 'Clade 6',
 'Clade 7',
 'Clade 8',
 'Clade 9',
 'Clade 10',
 'Clade 11',
 'Clade 12',
 'Clade 13',
 'Clade 14',
 'Clade 15',
 'Taxa',
 'Genus',
 'Species',
 'Key papers',
 'Specimen type',
 'Specimen No.',
 'Type of preservation',
 'Elements',
 'Skull',
 'Mandible',
 'Cervical vertebrae',
 'Dorsal vertebrae',
 'Sacral vertebrae',
 'Caudal vertebrae',
 'Chevrons',
 'Cervical.ribs',
 'Dorsal.ribs',
 'Scapulae',
 'Coracoids',
 'Sternum',
 'Humeri',
 'Ulnae',
 'Radii',
 'Carpals',
 'Metacarpals',
 'Manus.digits',
 'Ilia',
 'Pubes',
 'Ischia',
 'Femora',
 'Tibiae',
 'Fibulae',
 'Tarsals',
 'Metatarsals',
 'Pes.digits',
 'TOTAL']

We have the clade and subclade information from the summary dataframe, and I can use the taxa to look it up, so I can remove all the Clade columns. I can also remove species and genus (since they are in the taxa column) and the columns about papers and specimen type.

For the skeleton parts, I can refer back to the raw data from the source and the headings that I removed before importing this data. First, let's get rid of the columns we don't need.

In [53]:
#Drop unnecessary columns
completeness_df = completeness_df.drop(completeness_df.columns[0:18], axis = 1)
completeness_df = completeness_df.drop(completeness_df.columns[1:8], axis = 1)
#Remove any rows where Taxa is blank
completeness_df = completeness_df.dropna(subset = ['Taxa'])

In [54]:
completeness_df.head()

,Taxa,Skull,Mandible,Cervical vertebrae,Dorsal vertebrae,Sacral vertebrae,Caudal vertebrae,Chevrons,Cervical.ribs,Dorsal.ribs,...,Ilia,Pubes,Ischia,Femora,Tibiae,Fibulae,Tarsals,Metatarsals,Pes.digits,TOTAL
1,Aardonyx celestae,0.1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.1
2,Aardonyx celestae,0.133333333,0,0,0,0.2,0,0,0,0,...,0,0,0,0,0.25,0,0,0,0,3.583333333
3,Aardonyx celestae,0,0,0,0,10,0,0,0,0.1875,...,0,0,0,0,0,0,0,0,0,10.1875
4,Aardonyx celestae,0,0,0,0,0,0,0,0,0.1875,...,0,0,0,0,0,0,0,0,0,0.1875
5,Aardonyx celestae,0,0,0,0,0,0,0,0,0.1875,...,0,0,0,0,0,0,0,0,0,0.1875


Now I want to group the amounts found by part of the skeleton. According to the source data, the skeleton parts are:

- Skull
- Axial (Spine)
- Ribs
- Pectoral Girdle
- Forelimb
- Pelvic Girdle
- Hindlimb

The datatypes for all the measurements are currently objects, so I'll need to change them to floats to add them up.

In [55]:
#Get new list of columns
cols = list(completeness_df)
cols

['Taxa',
 'Skull',
 'Mandible',
 'Cervical vertebrae',
 'Dorsal vertebrae',
 'Sacral vertebrae',
 'Caudal vertebrae',
 'Chevrons',
 'Cervical.ribs',
 'Dorsal.ribs',
 'Scapulae',
 'Coracoids',
 'Sternum',
 'Humeri',
 'Ulnae',
 'Radii',
 'Carpals',
 'Metacarpals',
 'Manus.digits',
 'Ilia',
 'Pubes',
 'Ischia',
 'Femora',
 'Tibiae',
 'Fibulae',
 'Tarsals',
 'Metatarsals',
 'Pes.digits',
 'TOTAL']

In [56]:
#Create list of columns to change to float
float_cols = ['Skull',
 'Mandible',
 'Cervical vertebrae',
 'Dorsal vertebrae',
 'Sacral vertebrae',
 'Caudal vertebrae',
 'Chevrons',
 'Cervical.ribs',
 'Dorsal.ribs',
 'Scapulae',
 'Coracoids',
 'Sternum',
 'Humeri',
 'Ulnae',
 'Radii',
 'Carpals',
 'Metacarpals',
 'Manus.digits',
 'Ilia',
 'Pubes',
 'Ischia',
 'Femora',
 'Tibiae',
 'Fibulae',
 'Tarsals',
 'Metatarsals',
 'Pes.digits',
 'TOTAL']

In [57]:
#Replace '-' with 0 in dataframe
completeness_df = completeness_df.replace('-','0')

In [58]:
#Change float columns type to float
completeness_df[float_cols] = completeness_df[float_cols].astype('float')

In [59]:
#Check data types
completeness_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1139 entries, 1 to 1143
Data columns (total 29 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Taxa                1139 non-null   object 
 1   Skull               1139 non-null   float64
 2   Mandible            1139 non-null   float64
 3   Cervical vertebrae  1139 non-null   float64
 4   Dorsal vertebrae    1139 non-null   float64
 5   Sacral vertebrae    1139 non-null   float64
 6   Caudal vertebrae    1139 non-null   float64
 7   Chevrons            1139 non-null   float64
 8   Cervical.ribs       1139 non-null   float64
 9   Dorsal.ribs         1139 non-null   float64
 10  Scapulae            1139 non-null   float64
 11  Coracoids           1139 non-null   float64
 12  Sternum             1139 non-null   float64
 13  Humeri              1139 non-null   float64
 14  Ulnae               1139 non-null   float64
 15  Radii               1139 non-null   float64
 16  Carpal

In [60]:
#Create new summary columns for each skeleton category
completeness_df['skull'] = completeness_df.iloc[:, 1:3].sum(axis = 1)
completeness_df['vertebrae'] = completeness_df.iloc[:, 3:8].sum(axis = 1)
completeness_df['ribs'] = completeness_df.iloc[:, 8:10].sum(axis = 1)
completeness_df['pectorals'] = completeness_df.iloc[:, 10:13].sum(axis = 1)
completeness_df['forelimb'] = completeness_df.iloc[:, 13:19].sum(axis = 1)
completeness_df['pelvis'] = completeness_df.iloc[:, 19:22].sum(axis = 1)
completeness_df['hindlimb'] = completeness_df.iloc[:, 22:28].sum(axis = 1)
completeness_df.head()

,Taxa,Skull,Mandible,Cervical vertebrae,Dorsal vertebrae,Sacral vertebrae,Caudal vertebrae,Chevrons,Cervical.ribs,Dorsal.ribs,...,Metatarsals,Pes.digits,TOTAL,skull,vertebrae,ribs,pectorals,forelimb,pelvis,hindlimb
1,Aardonyx celestae,0.100000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0000,...,0.0,0.0,0.100000,0.100000,0.0,0.0000,0.0,0.0,0.0,0.00
2,Aardonyx celestae,0.133333,0.0,0.0,0.0,0.2,0.0,0.0,0.0,0.0000,...,0.0,0.0,3.583333,0.133333,0.2,0.0000,0.0,3.0,0.0,0.25
3,Aardonyx celestae,0.000000,0.0,0.0,0.0,10.0,0.0,0.0,0.0,0.1875,...,0.0,0.0,10.187500,0.000000,10.0,0.1875,0.0,0.0,0.0,0.00
4,Aardonyx celestae,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.1875,...,0.0,0.0,0.187500,0.000000,0.0,0.1875,0.0,0.0,0.0,0.00
5,Aardonyx celestae,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.1875,...,0.0,0.0,0.187500,0.000000,0.0,0.1875,0.0,0.0,0.0,0.00


In [61]:
#Drop columns for separate skeleton parts
completeness_df = completeness_df.drop(completeness_df.columns[1:28], axis = 1)
completeness_df.head()

,Taxa,TOTAL,skull,vertebrae,ribs,pectorals,forelimb,pelvis,hindlimb
1,Aardonyx celestae,0.100000,0.100000,0.0,0.0000,0.0,0.0,0.0,0.00
2,Aardonyx celestae,3.583333,0.133333,0.2,0.0000,0.0,3.0,0.0,0.25
3,Aardonyx celestae,10.187500,0.000000,10.0,0.1875,0.0,0.0,0.0,0.00
4,Aardonyx celestae,0.187500,0.000000,0.0,0.1875,0.0,0.0,0.0,0.00
5,Aardonyx celestae,0.187500,0.000000,0.0,0.1875,0.0,0.0,0.0,0.00


Now all that's left to do is reorder and rename the columns.

In [62]:
#Get list of columns
cols = list(completeness_df)
cols

['Taxa',
 'TOTAL',
 'skull',
 'vertebrae',
 'ribs',
 'pectorals',
 'forelimb',
 'pelvis',
 'hindlimb']

In [63]:
#Reorder columns
new_cols = ['Taxa',
 'skull',
 'vertebrae',
 'ribs',
 'pectorals',
 'forelimb',
 'pelvis',
 'hindlimb', 'TOTAL']

In [64]:
completeness_df = completeness_df[new_cols]

In [65]:
#Rename columns
final_cols = ['taxa',
 'skull',
 'vertebrae',
 'ribs',
 'pectorals',
 'forelimb',
 'pelvis',
 'hindlimb', 'total']

In [66]:
completeness_df.columns = [final_cols]
completeness_df.head()

,taxa,skull,vertebrae,ribs,pectorals,forelimb,pelvis,hindlimb,total
1,Aardonyx celestae,0.100000,0.0,0.0000,0.0,0.0,0.0,0.00,0.100000
2,Aardonyx celestae,0.133333,0.2,0.0000,0.0,3.0,0.0,0.25,3.583333
3,Aardonyx celestae,0.000000,10.0,0.1875,0.0,0.0,0.0,0.00,10.187500
4,Aardonyx celestae,0.000000,0.0,0.1875,0.0,0.0,0.0,0.00,0.187500
5,Aardonyx celestae,0.000000,0.0,0.1875,0.0,0.0,0.0,0.00,0.187500


## Exporting the Cleaned Data

Since this data took a lot of steps to clean, I will export it into new csvs and analyze it in a separate file.

In [67]:
#Reset index for both cleaned dataframes
summary_df = summary_df.reset_index(drop = True)
completeness_df = completeness_df.reset_index(drop = True)

In [63]:
#Export data
summary_df.to_csv('summary_df.csv', index = False)

In [64]:
completeness_df.to_csv('completeness_df.csv', index = False)